In [1]:
#When we train the model we should do data loading in tf records too

In [1]:
from PIL import Image
import numpy as np
import skimage.io as io
import tensorflow as tf

import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm_notebook, tnrange, tqdm

from scipy.signal import medfilt2d


/home/ubuntu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:

# Helper functions for defining tf types
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))



In [3]:
trainFolder = '../tgs-salt-identification-challenge/trainData/'
testFolder  = '../tgs-salt-identification-challenge/testData/'
depthFile = pd.read_csv('../depths.csv' )

In [4]:
%load_ext autoreload
%autoreload 2

import ImageReader

Using TensorFlow backend.


In [5]:
#get ids
train_ids = next(os.walk(trainFolder +"images"))[2]
test_ids = next(os.walk(testFolder +"images"))[2]

In [6]:
X_train, Y_train, depthVals = ImageReader.ReadSegmentationImages(trainFolder, depthFile)


  1%|          | 30/4000 [00:00<00:13, 290.84it/s]

Getting images and masks ... 


100%|██████████| 4000/4000 [00:13<00:00, 305.63it/s]


In [7]:
X_train = X_train/255.0#(X_train-X_train_mean)/(2*X_train_std)

In [8]:
#loading the final test set too
X_test, d_test = ImageReader.ReadSegmentationImages(testFolder, depthFile, readMasks = False)
X_test = X_test/255.0 #(X_test-X_train_mean)/(2*X_train_std)

  0%|          | 35/18000 [00:00<00:51, 347.11it/s]

Getting images and masks ... 


100%|██████████| 18000/18000 [00:52<00:00, 345.95it/s]


In [18]:
#train test splitting
valRatio = 0.04
nVal = int(len(X_train)*valRatio)
X_val = X_train[0:nVal]
Y_val = Y_train[0:nVal]
d_val = depthVals[0:nVal]

X_train0 = X_train[nVal:]
Y_train0 = Y_train[nVal:]
d_train0 = depthVals[nVal:]
X_train0.shape
print('{}, {}'.format(X_train0.shape, X_val.shape))

(3840, 101, 101, 1), (160, 101, 101, 1)


In [19]:
#Upsampling
X_train0=np.concatenate((X_train0,X_train0,X_train0,X_train0,X_train0),axis=0)
Y_train0=np.concatenate((Y_train0,Y_train0,Y_train0,Y_train0,Y_train0),axis=0)
d_train0=np.concatenate((d_train0,d_train0,d_train0,d_train0,d_train0),axis=0)

In [22]:
#now we have the train, validation and final test set, we write these 3 as 3 different tf records. 
#THe tf record has to entail: The image, the segmentation, the depth and a "has label", so we can easily train it together we should
#have an empty segmentation mask for the unlabeled ones. 

In [23]:
#in loop:
def write_segment_rec(image=X_train0,segment=Y_train0,is_train=1,depthfile=d_train0,file_name="salty_train.tfrecords"):
    writer = tf.python_io.TFRecordWriter(file_name)
    
    for j in tqdm(range(image.shape[0])):
        img=image[j,:,:,:]
        if is_train==1:
            annotation=segment[j,:,:,:]
        else:
            annotation=np.zeros(shape=(101,101,1),dtype=np.int8)
        height = img.shape[0]
        width = img.shape[1]
        depth=depthfile[j]
        img_raw = img.tostring()
        annotation_raw = annotation.tostring()
        example = tf.train.Example(features=tf.train.Features(feature={
                    'height': _int64_feature(height),
                    'width': _int64_feature(width),
                    'image_raw': _bytes_feature(img_raw),
                    'mask_raw': _bytes_feature(annotation_raw),
                    "depth":_float_feature(depth), 
                    "is_train":_float_feature(is_train)}
                                                              
                                                             ))
        writer.write(example.SerializeToString())
    writer.close()

In [24]:
write_segment_rec(image=X_train0,segment=Y_train0,is_train=1,depthfile=d_train0,file_name="salty_train.tfrecords")

100%|██████████| 19200/19200 [00:04<00:00, 4798.95it/s]


In [97]:
write_segment_rec(image=X_val,segment=Y_val,is_train=1,depthfile=d_val,file_name="salty_valid.tfrecords")

100%|██████████| 160/160 [00:00<00:00, 4740.93it/s]


In [98]:
write_segment_rec(image=X_test,segment=None,is_train=0,depthfile=d_test,file_name="salty_test.tfrecords")

100%|██████████| 18000/18000 [00:03<00:00, 4691.93it/s]
